<a href="https://colab.research.google.com/github/black-dalia/03_gettingAndCleaningData/blob/main/Desiree_modelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers, metrics
from tensorflow.keras.layers import Normalization
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import RobustScaler
import pandas as pd
import numpy as np 
import math
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import seaborn as sns
import datetime
from datetime import date, timedelta
#from clean import clean_all, get_data, get_bairros_data
#from preproc import get_format, get_popfile, clean_pop_data, extract_ts
idx = pd.IndexSlice

In [2]:
url = 'https://raw.githubusercontent.com/black-dalia/event-forecast-rio/modelling/preprocessed_data_1000.csv'

In [3]:
preprocessed_data_1000 = pd.read_csv(url)

In [4]:
# Preprocessing data
def subsample_sequence(data, length, AR): # Return a shorter dataframe with specified 
                                                 # length for a specific barrio
    last_possible = data.shape[0] - length
    random_start = np.random.randint(0, last_possible)
    data_sample = data[random_start: random_start+length]
    #data_sample = data_sample.loc[:,idx[:,AR]]
    data_sample = data_sample[AR]

    return data_sample

def split_subsample_sequence(data, length, AR): # Return a random sequence of specified length

    data_subsample = subsample_sequence(data, length, AR)
    y_sample = data_subsample.iloc[length-31:]
    
    X_sample = data_subsample[0:length-31]
    X_sample = X_sample.values
    return np.array(X_sample), np.array(y_sample)

def get_X_y(data, n_sequences, length, AR): # Return a sepcific number of (X,y) samples of specified length
                                                   # for a specified bairro

    X, y = [], []

    for i in range(n_sequences):
        (xi, yi) = split_subsample_sequence(data, length, AR)
        X.append(xi)
        y.append(yi)
        
    X = np.array(X)
    y = np.array(y)
    return X, y

def get_train_test(data,n_sequences,length, AR): # Return train and test data

    len_ = int(0.8*data.shape[0])
    data_train = data[:len_]
    data_test = data[len_:]
    
    test_seq = math.floor(n_sequences/4)
    
    X_train, y_train = get_X_y(data_train, n_sequences, length, AR)
    X_test, y_test = get_X_y(data_test, test_seq, length, AR)
    
    X_train = X_train.reshape(X_train.shape[0], X_train.shape[1],1)
    X_test = X_test.reshape(X_test.shape[0], X_test.shape[1],1)
    
    return X_train, y_train, X_test, y_test

In [5]:
AR_list = preprocessed_data_1000.columns.tolist()
all_data = {}

for AR in AR_list:
  data = get_train_test(preprocessed_data_1000, 2000, 200, AR)
  all_data[AR] = data

In [ ]:
AR_list

In [8]:
len(all_data)

30

In [9]:
all_data["Anchieta"][0].shape

(2000, 169, 1)

In [10]:
for AR in AR_list:
    normalizer = Normalization()
    normalizer.adapt(all_data[AR][0])
    normalizer.adapt(all_data[AR][2])

In [4]:
# Creating a model
def init_model(AR):
    metric = metrics.MAPE
    opt = optimizers.RMSprop(learning_rate=0.005)
    
    model = models.Sequential()
    model.add(layers.LSTM(30, return_sequences=True, activation='tanh'))
    model.add(layers.LSTM(10, activation='tanh'))
    model.add(layers.Dense(5, activation='relu'))
    model.add(layers.Dense(31, activation='linear'))
    
    model.compile(loss='mse', 
                  optimizer=opt, 
                  metrics=[metric])
    
    return model

# Parameters
# def get_parameters(): 
es = EarlyStopping(monitor='val_loss', verbose=1, patience=5, restore_best_weights=True)

# Fitting the model
def fit(model, AR):
    hist = model.fit(all_data[AR][0], all_data[AR][1],
            validation_split=0.3,
            epochs=5, 
            batch_size=32,
            callbacks=[es], verbose=0)
    return hist

# Evaluating the model 
def evaluate(hist, AR):
    results = hist.model.evaluate(all_data[AR][2], all_data[AR][3])
    return results

# All-in-1
def modelling(AR):
    results = evaluate(fit(init_model(AR), AR), AR)
    return results

In [14]:
model = init_model('Centro')
fitted_model = fit(model, 'Centro')

In [15]:
results = evaluate(fitted_model, 'Centro')
results

16/16 [==============================] - 1s 33ms/step - loss: 0.8432 - mean_absolute_percentage_error: 44.3881


[0.8431576490402222, 44.38812255859375]